# Test Query
Purpose is to perform a query against the index created.
This is only meant to test the retrieval portion of the RAG application.

## Required for this step
- Azure AI Search Service



In [1]:
# # Import required libraries  
import os  
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryType,
    VectorizedQuery,
)
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt 


In [6]:
#Load the configuration details for the Cognitive Search Service and Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault
config = json.load(open("config.json"))
    
# Azure AI Search Config
search_service_name = config["search_service_name"]
search_service_url = "https://{}.search.windows.net/".format(search_service_name)
search_admin_key = config["search_admin_key"]
index_name = config["search_index_name"]
index_schema_file = config["search_index_schema_file"]
search_api_version = config["search_api_version"]
search_headers = {  
    'Content-Type': 'application/json',  
    'api-key': search_admin_key  
}  

index_client = SearchIndexClient(
        endpoint=search_service_url, credential=AzureKeyCredential(search_admin_key))
search_client = SearchClient(endpoint=search_service_url, index_name=index_name, credential=AzureKeyCredential(search_admin_key))

print ('Search Service Name:', search_service_name)
print ('Index Name:', index_name)

#Azure OpenAI
api_base = config["openai_api_base"]
api_key = config["openai_api_key"]
openai_api_version = config["openai_api_version"]
embedding_model = config["openai_embedding_model"]

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_version=openai_api_version,
    azure_endpoint=api_base,
    api_key=api_key
)


In [3]:
# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return json.loads(response.model_dump_json())["data"][0]['embedding']

# generate_embeddings('test')

In [4]:
# Pure Hubrid Search
query = "how many shares were repurchased in the second quarter of 2022 by Microsoft?"  
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  
    print("==============================\n")

Title: "Corporate Share Repurchase Programs and Outcomes from 2016 to 2022"
Score: 0.03333333507180214
Content: On September 20, 2016, our Board of Directors approved a share repurchase program authorizing up to $40.0 billion in share repurchases. This share repurchase program commenced in December 2016 and was completed in February 2020.  
On September 18, 2019, our Board of Directors approved a share repurchase program authorizing up to $40.0 billion in share repurchases. This share repurchase program commenced in February 2020 and was completed in November 2021.  
On September 14, 2021, our Board of Directors approved a share repurchase program authorizing up to $60.0 billion in share repurchases. This share repurchase program commenced in November 2021, following completion of the program approved on September 18, 2019, has no expiration date, and may be terminated at any time. As of June 30, 2022, $40.7 billion remained of this $60.0 billion share repurchase program.  
<!-- PageNu

In [5]:
# Pure Hybrd+Semantic Search
query = "how many shares were repurchased in the second quarter of 2022 by Microsoft?"  
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='vector-semantic-configuration',
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  
    print("==============================\n")

Title: "Corporate Share Repurchase Programs and Outcomes from 2016 to 2022"
Score: 0.03333333507180214
Content: On September 20, 2016, our Board of Directors approved a share repurchase program authorizing up to $40.0 billion in share repurchases. This share repurchase program commenced in December 2016 and was completed in February 2020.  
On September 18, 2019, our Board of Directors approved a share repurchase program authorizing up to $40.0 billion in share repurchases. This share repurchase program commenced in February 2020 and was completed in November 2021.  
On September 14, 2021, our Board of Directors approved a share repurchase program authorizing up to $60.0 billion in share repurchases. This share repurchase program commenced in November 2021, following completion of the program approved on September 18, 2019, has no expiration date, and may be terminated at any time. As of June 30, 2022, $40.7 billion remained of this $60.0 billion share repurchase program.  
<!-- PageNu